In [2]:
try:
    from ultralytics import YOLO
except:
    %pip install -q ultralytics

## 0. Imports and agnostic code

In [ ]:
import torch
import cv2 as cv
import numpy as np
from pathlib import Path
import time
device = "cpu"
device

'cpu'

## 1. Object detection using YOLO11

In [11]:
model = YOLO("yolo11n.pt")

# Train the model using the 'coco8.yaml' dataset for 3 epochs
# results = model.train(data="coco.yaml", epochs=3)

# Evaluate the model's performance on the validation set
# results = model.val()

# Perform object detection on an image using the model
# results = model("https://ultralytics.com/images/bus.jpg")

# Export the model to ONNX format
# success = model.export(format="onnx")

img_path = Path("pics\dog.png")
results = model(img_path)
results[0].show()


image 1/1 c:\Users\jayat\projects\object identification with cnn\pics\dog.png: 640x480 1 dog, 196.6ms
Speed: 11.5ms preprocess, 196.6ms inference, 14.7ms postprocess per image at shape (1, 3, 640, 480)


## 2. Integrating OpenCV

In [ ]:
# Trackers
seen_objects = {}    # {label: first_seen_time}
stable_objects = set()    # objects seen after 30 seconds

# Live feed capture from port 0 (in-built webcam)
cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    # Run YOLO detection
    results = model(frame)
    boxes = results[0].boxes
    detected_labels = []

    # Extract detected class names
    if boxes is not None:
        for c in boxes.cls:
            label = model.names[int(c)]
            detected_labels.append(label)

    # Track objects
    current_time = time.time()
    for label in detected_labels:
        if label not in seen_objects:
            seen_objects[label] = current_time  # first seen
        else:
            if (current_time - seen_objects[label] > 30) and (label not in stable_objects):
                stable_objects.add(label)
                # speak(label)
    
    # Reset if object disappears
    for label in list(seen_objects.keys()):
        if label not in detected_labels:
            seen_objects.pop(label)
            stable_objects.discard(label)

    # Show annotated frame
    annotated_frame = results[0].plot()
    cv.imshow("YOLO Live Feed", annotated_frame)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv.destroyAllWindows()

In [13]:
results

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

In [15]:
results[0].boxes

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0., 0., 0., 0.])
conf: tensor([0.5470, 0.4729, 0.4045, 0.2511])
data: tensor([[2.2241e+02, 2.8653e+02, 6.3871e+02, 4.7960e+02, 5.4696e-01, 0.0000e+00],
        [4.2605e+02, 1.3574e+02, 6.3951e+02, 4.7938e+02, 4.7292e-01, 0.0000e+00],
        [2.2357e+02, 1.3052e+02, 6.3413e+02, 4.7870e+02, 4.0452e-01, 0.0000e+00],
        [8.0767e+00, 3.1055e+02, 4.6079e+02, 4.7948e+02, 2.5106e-01, 0.0000e+00]])
id: None
is_track: False
orig_shape: (480, 640)
shape: torch.Size([4, 6])
xywh: tensor([[430.5599, 383.0649, 416.2933, 193.0662],
        [532.7832, 307.5628, 213.4586, 343.6426],
        [428.8454, 304.6066, 410.5607, 348.1816],
        [234.4332, 395.0142, 452.7130, 168.9332]])
xywhn: tensor([[0.6727, 0.7981, 0.6505, 0.4022],
        [0.8325, 0.6408, 0.3335, 0.7159],
        [0.6701, 0.6346, 0.6415, 0.7254],
        [0.3663, 0.8229, 0.7074, 0.3519]])
xyxy: tensor([[222.4132, 286.5318, 638.7065, 479.5980],
        [426.0539